<h2><center><b>Lidar Car Detection</b></center></h2>

![](https://images.aicrowd.com/raw_images/challenges/banner_file/903/c5e145e1adaa2493032a.jpg)


### Things implemented

  - Working of LIDAR system 
  - Usage of [scikit-learn](https://scikit-learn.org/stable/) for binary classification.

Note : Create a copy of the notebook and use the copy for submission. Go to File > Save a Copy in Drive to create a new copy



# Downloading Dataset

Installing `aicrowd-cli`

In [1]:
%pip install aicrowd-cli
%load_ext aicrowd.magic

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
%aicrowd login

Please login here: https://api.aicrowd.com/auth/mqo6T_boxx5nOeLXEd7Qg3Thr5r9F0C5d8z_6n8eJDU


ATTENTION: default value of option mesa_glthread overridden by environment.
ATTENTION: default value of option mesa_glthread overridden by environment.
ATTENTION: default value of option mesa_glthread overridden by environment.
ATTENTION: default value of option mesa_glthread overridden by environment.
Missing chrome or resource URL: resource://gre/modules/UpdateListener.jsm
Missing chrome or resource URL: resource://gre/modules/UpdateListener.sys.mjs


API Key valid
Gitlab access token valid
Saved details successfully!


In [3]:
!rm -rf data
!mkdir data
%aicrowd ds dl -c lidar-car-detection -o data

test.npz:   0%|          | 0.00/163M [00:00<?, ?B/s]

train.npz:   0%|          | 0.00/108M [00:00<?, ?B/s]

Missing chrome or resource URL: resource://gre/modules/UpdateListener.jsm
Missing chrome or resource URL: resource://gre/modules/UpdateListener.sys.mjs


# Importing Libraries

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import random

# Reading the dataset

In [18]:
# Reading the training dataset

train_data = np.load("../lidar-car-detection/data/train.npz", allow_pickle=True)
train_data = train_data['train']

train_data.shape

(400, 2)

### Visualizing the dataset

In this section, we will be visualizing a sample 3D lidar data

In [19]:
# Getting a random 3D lidar sample data
INDEX = random.randint(0, train_data.shape[0])

# Getting the individual x,y and z points.
x = train_data[INDEX][0][:, 0].tolist()
y = train_data[INDEX][0][:, 1].tolist()
z = train_data[INDEX][0][:, 2].tolist()

# Label for the corrosponding sample ( no. of cars )
label  = train_data[INDEX][1]

# Generating the 3D graph
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers',
                                   marker=dict(
                                   size=1,       
                                   colorscale='Viridis',
                                   opacity=0.8))])
print("No. of cars : ", label)
fig.show()

No. of cars :  1


## Splitting the dataset

In [20]:
# Getting the 3d points and flattening the points into 1d array ( using only 100 training samples for faster teaining )
X = train_data[:100, 0]
X = [i.flatten()    for i in X]

# labels
y = train_data[:100, 1]

In [21]:
# Splitting the dataset into training and testing
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Training the model

In [22]:
model = RandomForestRegressor(verbose=True, n_jobs=-1)

In [23]:
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   17.5s finished


RandomForestRegressor(n_jobs=-1, verbose=True)

# Validation 

In [24]:
model.score(X_val, y_val)

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


0.06083179012345685

# Generating the predictions

In [25]:
# Loading the test data

test_data = np.load("../lidar-car-detection/data/test.npz", allow_pickle=True)
test_data = test_data['test']

test_data.shape

(601,)

In [26]:
# flattening the points into 1d array
X_test = X = [i.flatten()    for i in test_data]

In [27]:
# Generating the predictions
predictions = model.predict(X_test)
predictions.shape

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


(601,)

In [28]:
submission = pd.DataFrame({"label":predictions})
submission

,label
0,2.10
1,2.64
2,2.28
3,2.58
4,2.24
...,...
596,1.52
597,2.22
598,2.72
599,1.76


In [29]:
# Saving the predictions
!rm -rf assets
!mkdir assets
submission.to_csv(os.path.join("assets", "submission.csv"))